In [1]:

import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["PYTHONHASHSEED"] = "42"

import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import f1_score


# ------------------------------------------------------------------------------
# 0) Utils
# ------------------------------------------------------------------------------
def set_seed(seed: int, deterministic: bool = True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.use_deterministic_algorithms(True, warn_only=False)


def make_seed_worker(seed: int):
    def seed_worker(worker_id: int):
        worker_seed = seed + worker_id
        np.random.seed(worker_seed)
        random.seed(worker_seed)
    return seed_worker


def count_params(model, trainable_only=True):
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    return sum(p.numel() for p in model.parameters())


# ------------------------------------------------------------------------------
# 1) Dataset
# ------------------------------------------------------------------------------
class UCIHARDataset(Dataset):
    def __init__(self, data_path, split='train'):
        self.split = split

        if split == 'train':
            y = np.loadtxt(os.path.join(data_path, 'train', 'y_train.txt'))
            signal_path = os.path.join(data_path, 'train', 'Inertial Signals')
        else:
            y = np.loadtxt(os.path.join(data_path, 'test', 'y_test.txt'))
            signal_path = os.path.join(data_path, 'test', 'Inertial Signals')

        signals = []
        signal_files = [
            "total_acc_x", "total_acc_y", "total_acc_z",
            "body_acc_x", "body_acc_y", "body_acc_z",
            "body_gyro_x", "body_gyro_y", "body_gyro_z",
        ]

        for signal_file in signal_files:
            filename = os.path.join(signal_path, f'{signal_file}_{split}.txt')
            signals.append(np.loadtxt(filename))

        # X: (N, T, C)
        self.X = np.stack(signals, axis=-1).astype(np.float32)
        self.y = (y - 1).astype(np.int64)

        print(f"[{split}] X: {self.X.shape}, y: {self.y.shape}")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.from_numpy(self.X[idx]).float()     # (T, C)
        x = x.permute(1, 0).contiguous()              # (C, T)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return x, y


def fit_transform_scaler_train_only(train_X, test_X):
    N, C, T = train_X.shape

    train_flat = train_X.transpose(0, 2, 1).reshape(-1, C)  # (N*T, C)
    test_flat  = test_X.transpose(0, 2, 1).reshape(-1, C)

    scaler = RobustScaler()
    scaler.fit(train_flat)

    train_scaled = scaler.transform(train_flat).reshape(N, T, C).transpose(0, 2, 1)
    test_scaled  = scaler.transform(test_flat).reshape(test_X.shape[0], T, C).transpose(0, 2, 1)

    return train_scaled.astype(np.float32), test_scaled.astype(np.float32)


class ArrayDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()   # (N, C, T)
        self.y = torch.from_numpy(y).long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# ------------------------------------------------------------------------------
# 2) Model
# ------------------------------------------------------------------------------
class RelativeEnergyPhysics(nn.Module):
    def __init__(self, acc_indices, gyro_indices):
        super().__init__()
        self.acc_indices = acc_indices
        self.gyro_indices = gyro_indices
        self.m = nn.Parameter(torch.tensor(1.0), requires_grad=True)
        self.I = nn.Parameter(torch.tensor(1.0), requires_grad=True)

    def forward(self, x):  # x: (B, C, T)
        m_pos = F.softplus(self.m)
        I_pos = F.softplus(self.I)

        x_t = x.transpose(1, 2)  # (B, T, C)
        B, T, _ = x_t.shape

        # accel proxy
        if len(self.acc_indices) > 0:
            acc_data = x_t[:, :, self.acc_indices]
            n_acc = len(self.acc_indices) // 3
            if n_acc > 0:
                acc_reshaped = acc_data.view(B, T, n_acc, 3)
                acc_mag = (acc_reshaped ** 2).sum(dim=-1).mean(dim=-1, keepdim=True)  # (B,T,1)
            else:
                acc_mag = torch.zeros(B, T, 1, device=x.device)
        else:
            acc_mag = torch.zeros(B, T, 1, device=x.device)

        E_kin = 0.5 * m_pos * acc_mag

        # gyro proxy
        if len(self.gyro_indices) > 0:
            gyro_data = x_t[:, :, self.gyro_indices]
            n_gyro = len(self.gyro_indices) // 3
            if n_gyro > 0:
                gyro_reshaped = gyro_data.view(B, T, n_gyro, 3)
                gyro_mag = (gyro_reshaped ** 2).sum(dim=-1).mean(dim=-1, keepdim=True)  # (B,T,1)
            else:
                gyro_mag = torch.zeros(B, T, 1, device=x.device)
        else:
            gyro_mag = torch.zeros(B, T, 1, device=x.device)

        E_rot = 0.5 * I_pos * gyro_mag
        return E_kin + E_rot  # (B,T,1)


class PotentialEnergyField(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.energy_net = nn.Sequential(
            nn.Linear(input_dim + 1, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x, E_phys):  # x:(B,C,T), E_phys:(B,T,1)
        x_t = x.transpose(1, 2)                # (B,T,C)
        x_aug = torch.cat([x_t, E_phys], -1)   # (B,T,C+1)
        return self.energy_net(x_aug)          # (B,T,1)


class EnergyGradientFlow(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.gradient_net = nn.Sequential(
            nn.Linear(input_dim + 1, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x, E_phys):  # x:(B,C,T), E_phys:(B,T,1)
        x_t = x.transpose(1, 2)                # (B,T,C)
        x_aug = torch.cat([x_t, E_phys], -1)   # (B,T,C+1)
        return self.gradient_net(x_aug)        # (B,T,C)


class RateAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.rate_proj = nn.Linear(hidden_dim, hidden_dim)
        self.rate_gate = nn.Linear(hidden_dim, 1)

    def forward(self, features, energy):  # features:(B,T,H), energy:(B,T,1)
        energy_rate = torch.cat([energy[:, :1, :], energy[:, 1:, :] - energy[:, :-1, :]], dim=1)
        ctx = self.rate_proj(features)
        scores = self.rate_gate(ctx + energy_rate)
        attn = torch.softmax(scores.squeeze(-1), dim=1)
        return attn, features * attn.unsqueeze(-1)


class PhaseAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.phase_proj = nn.Linear(hidden_dim, hidden_dim)
        self.phase_gate = nn.Linear(hidden_dim, 1)

    def forward(self, features, energy):
        energy_pad = F.pad(energy.transpose(1, 2), (1, 1), mode='replicate').transpose(1, 2)
        curv = energy_pad[:, 2:, :] - 2 * energy_pad[:, 1:-1, :] + energy_pad[:, :-2, :]
        ctx = self.phase_proj(features)
        scores = self.phase_gate(ctx + curv)
        attn = torch.softmax(scores.squeeze(-1), dim=1)
        return attn, features * attn.unsqueeze(-1)


class AttentionToAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.meta_attention = nn.Sequential(
            nn.Linear(2, hidden_dim // 4),
            nn.GELU(),
            nn.Linear(hidden_dim // 4, 2),
            nn.Softmax(dim=-1)
        )
        self.conflict_detector = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, rate_attn, phase_attn, rate_feat, phase_feat):
        attn_stack = torch.stack([rate_attn, phase_attn], dim=-1)  # (B,T,2)
        reliability = self.meta_attention(attn_stack)              # (B,T,2)
        rate_reliability = reliability[:, :, 0:1]
        phase_reliability = reliability[:, :, 1:2]

        feat_cat = torch.cat([rate_feat, phase_feat], dim=-1)
        conflict_score = self.conflict_detector(feat_cat).mean(dim=1).squeeze(-1)

        reconciled = rate_reliability * rate_feat + phase_reliability * phase_feat
        return reconciled, conflict_score


class LandscapeGeometryEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        combined_dim = input_dim * 2 + 1
        self.input_proj = nn.Conv1d(combined_dim, hidden_dim, kernel_size=1)
        self.conv = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=5, padding=2)
        self.norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(0.15)
        self.temporal_attn = nn.MultiheadAttention(hidden_dim, num_heads=4, batch_first=True)

        self.rate_attention = RateAttention(hidden_dim)
        self.phase_attention = PhaseAttention(hidden_dim)
        self.a2 = AttentionToAttention(hidden_dim)

    def forward(self, energy, gradient, x_original):
        x_t = x_original.transpose(1, 2)                     # (B,T,C)
        state = torch.cat([x_t, gradient, energy], dim=-1)   # (B,T,2C+1)

        h = self.input_proj(state.transpose(1, 2))           # (B,H,T)
        h = self.conv(h)
        h = self.norm(h.transpose(1, 2)).transpose(1, 2)
        h = F.gelu(h)
        h = self.dropout(h)

        h_t = h.transpose(1, 2)                              # (B,T,H)
        h_t, _ = self.temporal_attn(h_t, h_t, h_t)

        rate_attn, rate_feat = self.rate_attention(h_t, energy)
        phase_attn, phase_feat = self.phase_attention(h_t, energy)
        reconciled, conflict = self.a2(rate_attn, phase_attn, rate_feat, phase_feat)
        return reconciled, conflict


class MIELHAR_A2(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, sensor_config,
                 physics_grad_weight=0.1, energy_reg_weight=0.01):
        super().__init__()
        self.acc_indices = sensor_config["acc_indices"]
        self.gyro_indices = sensor_config["gyro_indices"]

        self.physics = RelativeEnergyPhysics(self.acc_indices, self.gyro_indices)
        self.potential_field = PotentialEnergyField(input_dim, hidden_dim)
        self.gradient_flow   = EnergyGradientFlow(input_dim, hidden_dim)
        self.physics_grad_weight = physics_grad_weight

        self.encoder = LandscapeGeometryEncoder(input_dim, hidden_dim)
        self.energy_reg_weight = energy_reg_weight

        self.pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, num_classes),
        )

    def _physics_grad(self, x):
        x_t = x.transpose(1, 2)  # (B,T,C)
        m_pos = F.softplus(self.physics.m)
        I_pos = F.softplus(self.physics.I)

        pg = torch.zeros_like(x_t)
        if len(self.acc_indices) > 0:
            pg[:, :, self.acc_indices] = x_t[:, :, self.acc_indices] * m_pos
        if len(self.gyro_indices) > 0:
            pg[:, :, self.gyro_indices] = x_t[:, :, self.gyro_indices] * I_pos
        return pg

    def forward(self, x, return_energy=False):
        E_phys = self.physics(x)  # (B,T,1)

        energy = self.potential_field(x, E_phys)        # (B,T,1)
        grad   = self.gradient_flow(x, E_phys)          # (B,T,C)
        grad = grad + self.physics_grad_weight * self._physics_grad(x)

        landscape_features, _ = self.encoder(energy, grad, x)  # (B,T,H)

        g = self.pool(landscape_features.transpose(1, 2)).squeeze(-1)  # (B,H)
        logits = self.classifier(g)

        if return_energy:
            return logits, energy, grad
        return logits


def compute_energy_loss(energy, gradient, beta_gradmag=0.1):
    energy_diff = energy[:, 1:] - energy[:, :-1]
    smoothness = torch.mean(energy_diff ** 2)
    grad_mag = torch.norm(gradient, dim=-1)
    grad_loss = torch.mean(grad_mag)
    return smoothness + beta_gradmag * grad_loss


# ------------------------------------------------------------------------------
# 3) Train/Eval
# ------------------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, criterion, device, beta_gradmag=0.1):
    model.train()

    total_sum = 0.0
    cls_sum = 0.0
    e_sum = 0.0
    n_batches = 0

    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        logits, energy, grad = model(x, return_energy=True)
        cls_loss = criterion(logits, y)
        e_loss = compute_energy_loss(energy, grad, beta_gradmag=beta_gradmag)

        loss = cls_loss + model.energy_reg_weight * e_loss

        loss.backward()
        optimizer.step()

        total_sum += loss.item()
        cls_sum += cls_loss.item()
        e_sum += e_loss.item()
        n_batches += 1

    return {
        "loss_total": total_sum / max(1, n_batches),
        "loss_cls": cls_sum / max(1, n_batches),
        "loss_energy": e_sum / max(1, n_batches),
    }


@torch.no_grad()
def eval_macro_f1(model, loader, device):
    model.eval()
    all_preds, all_labels = [], []
    for x, y in loader:
        x = x.to(device, non_blocking=True)
        logits = model(x)
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds.cpu().numpy())
        all_labels.append(y.numpy())
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    return f1_score(all_labels, all_preds, average='macro')


# ------------------------------------------------------------------------------
# 4) Main
# ------------------------------------------------------------------------------
def main():
    CFG = {
        "seed": 42,
        "deterministic": True,

        "dataset_root": "/content/drive/MyDrive/Colab Notebooks/HAR_data/UCI_HAR",

        "batch_size": 512,
        "num_workers": 2,
        "pin_memory": True,

        "epochs": 100,
        "lr": 8e-4,
        "weight_decay": 3e-4,
        "label_smoothing": 0.05,
        "physics_grad_weight": 0.25,  # EnergyGradientFlow: grad + alpha * physics_grad_weight

        "acc_indices": [0, 1, 2],
        "gyro_indices": [6, 7, 8],

        # loss weights
        "w_energy_reg": 0.1,      # multiply energy loss
        "beta_gradmag": 0.2,      # compute_energy_loss: smooth + beta * gradmag

        # model
        "hidden_dim": 64,
        "num_classes": 6,
        "input_dim": 9,

        # logging
        "log_every": 10,
    }

    set_seed(CFG["seed"], CFG["deterministic"])
    seed_worker = make_seed_worker(CFG["seed"])
    g = torch.Generator()
    g.manual_seed(CFG["seed"])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("=" * 80)
    print("MIEL-HAR with Attention-to-Attention (A²)")
    print("=" * 80)
    print(f"Device: {device}\n")

    # 1) Load official splits
    train_raw = UCIHARDataset(CFG["dataset_root"], split="train")
    test_raw  = UCIHARDataset(CFG["dataset_root"], split="test")

    X_train = np.transpose(train_raw.X, (0, 2, 1)).astype(np.float32)  # (N,C,T)
    y_train = train_raw.y.astype(np.int64)

    X_test = np.transpose(test_raw.X, (0, 2, 1)).astype(np.float32)
    y_test = test_raw.y.astype(np.int64)

    # 2) Standardize train-only
    X_train_s, X_test_s = fit_transform_scaler_train_only(X_train, X_test)

    train_ds = ArrayDataset(X_train_s, y_train)
    test_ds  = ArrayDataset(X_test_s, y_test)

    train_loader = DataLoader(
        train_ds,
        batch_size=CFG["batch_size"],
        shuffle=True,
        num_workers=CFG["num_workers"],
        pin_memory=CFG["pin_memory"] and (device.type == "cuda"),
        worker_init_fn=seed_worker,
        generator=g,
        drop_last=False,
        persistent_workers=(CFG["num_workers"] > 0),
    )
    test_loader = DataLoader(
        test_ds,
        batch_size=CFG["batch_size"],
        shuffle=False,
        num_workers=CFG["num_workers"],
        pin_memory=CFG["pin_memory"] and (device.type == "cuda"),
        worker_init_fn=seed_worker,
        generator=g,
        drop_last=False,
        persistent_workers=(CFG["num_workers"] > 0),
    )

    sensor_config = {"acc_indices": CFG["acc_indices"], "gyro_indices": CFG["gyro_indices"]}

    model = MIELHAR_A2(
        input_dim=CFG["input_dim"],
        hidden_dim=CFG["hidden_dim"],
        num_classes=CFG["num_classes"],
        sensor_config=sensor_config,
        physics_grad_weight=CFG["physics_grad_weight"],
        energy_reg_weight=CFG["w_energy_reg"],
    ).to(device)

    n_params = count_params(model, trainable_only=True)
    print(f"Parameters: {n_params:,} ({n_params/1e6:.3f} M)")

    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])

    warmup_epochs = 5
    warmup_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer,
        start_factor=0.1,
        end_factor=1.0,
        total_iters=warmup_epochs
    )

    criterion = nn.CrossEntropyLoss(label_smoothing=CFG["label_smoothing"])

    best_test_f1 = -1.0
    best_state = None
    best_epoch = -1

    print("-" * 100)
    print(f"{'Epoch':<8} {'LR':<10} {'Total':<10} {'Cls':<10} {'E':<10} {'TestF1(macro)':<14} {'Best':<6}")
    print("-" * 100)

    for epoch in range(1, CFG["epochs"] + 1):
        train_logs = train_one_epoch(
            model=model,
            loader=train_loader,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            beta_gradmag=CFG["beta_gradmag"],
        )

        if epoch <= warmup_epochs:
            warmup_scheduler.step()
        lr = optimizer.param_groups[0]["lr"]

        test_f1 = eval_macro_f1(model, test_loader, device)

        is_best = ""
        if test_f1 > best_test_f1 + 1e-12:
            best_test_f1 = test_f1
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            best_epoch = epoch
            is_best = "*"

        if (epoch % CFG["log_every"]) == 0:
            print(f"{epoch:<8} {lr:<10.6f} "
                  f"{train_logs['loss_total']:<10.4f} {train_logs['loss_cls']:<10.4f} {train_logs['loss_energy']:<10.4f} "
                  f"{test_f1:<14.4f} {is_best:<6}")

    print("-" * 80)
    print(f"Best Test F1(macro): {best_test_f1:.4f} @ epoch {best_epoch}")

    # final confirm
    model.load_state_dict(best_state)
    final_test_f1 = eval_macro_f1(model, test_loader, device)

    print("\nFinal Test:")
    print(f"Test F1(macro): {final_test_f1:.4f}")
    print("=" * 80)


if __name__ == "__main__":
    main()

MIEL-HAR with Attention-to-Attention (A²)
Device: cuda

[train] X: (7352, 128, 9), y: (7352,)
[test] X: (2947, 128, 9), y: (2947,)
Parameters: 71,015 (0.071 M)
----------------------------------------------------------------------------------------------------
Epoch    LR         Total      Cls        E          TestF1(macro)  Best  
----------------------------------------------------------------------------------------------------
10       0.000800   0.4275     0.4054     0.2216     0.9278         *     
20       0.000800   0.3005     0.2865     0.1400     0.9386               
30       0.000800   0.2807     0.2685     0.1225     0.9434               
40       0.000800   0.2656     0.2547     0.1086     0.9423               
50       0.000800   0.2711     0.2610     0.1008     0.9400               
60       0.000800   0.2567     0.2476     0.0913     0.9464               
70       0.000800   0.2557     0.2471     0.0860     0.9497               
80       0.000800   0.2565     0.2486 